In [2]:
import utils 
import os, sys, shutil, random
import numpy as np
from IPython.display import display, Image
from IPython import display
import yaml
import glob
import cv2
display.clear_output()
!yolo checks
HOME = os.getcwd()

Ultralytics YOLOv8.2.42 🚀 Python-3.8.10 torch-2.2.1+cu121 CUDA:0 (Quadro RTX 8000, 48585MiB)
Setup complete ✅ (72 CPUs, 250.5 GB RAM, 1870.7/1876.2 GB disk)

OS                  Linux-5.15.0-107-generic-x86_64-with-glibc2.29
Environment         Linux
Python              3.8.10
Install             pip
RAM                 250.54 GB
CPU                 Intel Xeon Gold 5220 2.20GHz
CUDA                12.1

numpy               ✅ 1.24.4<2.0.0,>=1.23.5
matplotlib          ✅ 3.7.5>=3.3.0
opencv-python       ✅ 4.8.0.74>=4.6.0
pillow              ✅ 10.2.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.10.1>=1.4.1
torch               ✅ 2.2.1>=1.8.0
torchvision         ✅ 0.17.1>=0.9.0
tqdm                ✅ 4.66.2>=4.64.0
psutil              ✅ 5.9.8
py-cpuinfo          ✅ 9.0.0
pandas              ✅ 2.0.3>=1.1.4
seaborn             ✅ 0.13.2>=0.11.0
ultralytics-thop    ✅ 2.0.0>=2.0.0


In [ ]:
def split_data(input_dir:str, output_dir:str, val_ratio=0.2) -> None:
    utils.create_dir_structure(output_dir)
    
    train_images_dir = os.path.join(input_dir, 'images')
    print(f"train_images_dir: {train_images_dir}")
    train_masks_dir = os.path.join(input_dir, 'masks')
    print(f"train_masks_dir: {train_masks_dir}")
    
    categories = ['Arch', 'Blackberry', 'Round']
    
    for category in categories:
        mask_category_dir = os.path.join(train_masks_dir, category)
        print(f"Processing category: {category}")
        
       
        image_files = sorted(os.listdir(train_images_dir))
        image_files = [f for f in image_files if f.lower().endswith(('tif'))]  # Make the extension check case-insensitive
        
        num_images = len(image_files)
        num_val = int(num_images * val_ratio)
        
        val_images = random.sample(image_files, num_val)
        train_images = [img for img in image_files if img not in val_images]
        
        for img in train_images:
            img_name = os.path.basename(img)
            src_mask_path = os.path.join(mask_category_dir, img)
            dst_mask_path = os.path.join(output_dir, 'train_masks', category, img)
            
            shutil.copy(src_mask_path, dst_mask_path)
        
        for img in val_images:
            img_name = os.path.basename(img)
            src_mask_path = os.path.join(mask_category_dir, img)
            dst_mask_path = os.path.join(output_dir, 'val_masks', category, img)

            shutil.copy(src_mask_path, dst_mask_path)

    print(f"Category {category} - Train: {len(train_images)}, Val: {len(val_images)}")

    for img in train_images:
        img_name = os.path.basename(img)
        src_img_path = os.path.join(train_images_dir, img_name)
        dst_img_path = os.path.join(output_dir, 'train_images', img_name)
        shutil.copy(src_img_path, dst_img_path)

    for img in val_images:
            img_name = os.path.basename(img)
            src_img_path = os.path.join(train_images_dir, img_name)   
            dst_img_path = os.path.join(output_dir, 'val_images', img_name)
            shutil.copy(src_img_path, dst_img_path)

        


        

In [ ]:
input_directory = "train"
output_directory = HOME + "/input"
split_data(input_directory, output_directory)

In [ ]:
def rename_double_extension_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.tif.tif'):
            new_filename = filename.replace('.tif.tif', '.tif')
            os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
            print(f"Renamed: {filename} to {new_filename}")


def remove_dm3_extension(directory):
    for filename in os.listdir(directory):
        # Check if the filename contains '.dm3'
        if '.dm3' in filename:
            # Create the new filename by removing '.dm3'
            new_filename = filename.replace('.dm3', '')
            # Construct full file paths
            old_file_path = os.path.join(directory, filename)
            new_file_path = os.path.join(directory, new_filename)
            # Rename the file
            os.rename(old_file_path, new_file_path)
            print(f"Renamed: {filename} to {new_filename}")


In [ ]:
# Example usage
directory_path = HOME +  "/input/val_masks/Blackberry"
remove_dm3_extension(directory_path)

In [ ]:
import os
import glob
import cv2
import json
import numpy as np
import shutil
import utils


def process_masks(mask_paths: str, output_dir: str) -> None:
    category_ids = {
        "Arch": 1,
        "Blackberry": 2,
        "Round": 3
    }

    annotations = []
    images = []
    image_id = 0
    ann_id = 0
    MASKS_EXT = 'tif'
    ORIGINAL_EXT = 'tif'

    for category in category_ids.keys():
        for mask_image in glob.glob(os.path.join(mask_paths, category, f'*.{MASKS_EXT}')):
            img_file_name = f'{os.path.basename(mask_image).split(".")[0]}.{ORIGINAL_EXT}'
            mask = cv2.imread(mask_image, cv2.IMREAD_UNCHANGED)
            
            if mask is None:
                print(f"Warning: Mask image {mask_image} could not be read.")
                continue
            
            # Get image dimensions
            height, width = mask.shape[:2]

            # Create or find existing image annotation
            if img_file_name not in map(lambda img: img['file_name'], images):
                image_id += 1
                images.append({
                    "id": image_id,
                    "file_name": img_file_name,
                    "height": height,
                    "width": width
                })
                image = images[-1]  # The newly added image
            else:
                image = [element for element in images if element['file_name'] == img_file_name][0]

            unique_values = np.unique(mask)
            for value in unique_values:
                if value == 0:
                    continue

                object_mask = (mask == value).astype(np.uint8) * 255
                polygons = utils.mask_to_polygons(object_mask)

                for poly in polygons:
                    ann_id += 1
                    annotations.append({
                        "id": ann_id,
                        "image_id": image['id'],
                        "category_id": category_ids[category],
                        "segmentation": [poly],
                        "area": cv2.contourArea(np.array(poly).reshape(-1, 2)),
                        "bbox": list(cv2.boundingRect(np.array(poly).reshape(-1, 2))),
                        "iscrowd": 0
                    })

    coco_output = {
        "images": images,
        "annotations": annotations,
        "categories": [{"id": value, "name": key, "supercategory": key} for key, value in category_ids.items()]
    }

    with open(os.path.join(output_dir, 'coco_annotations.json'), 'w') as f:
        json.dump(coco_output, f)

    print("Created %d annotations for images in folder: %s" % (len(annotations), mask_paths))


In [ ]:
HOME
train_masks_path= HOME+  "/input/train_masks"
train_output_dir =HOME + "/input/train_images"
process_masks(mask_paths=train_masks_path, output_dir=train_output_dir)

In [ ]:
val_masks_path= HOME+  "/input/val_masks"
val_output_dir =HOME + "/input/val_images"
process_masks(mask_paths=val_masks_path, output_dir=val_output_dir)

In [ ]:
def convert_to_yolo(input_images_path, input_json_path, output_images_path, output_labels_path):
    # Open JSON file containing image annotations
    f = open(input_json_path)
    data = json.load(f)
    f.close()

    # Create directories for output images and labels
    os.makedirs(output_images_path, exist_ok=True)
    os.makedirs(output_labels_path, exist_ok=True)

    # List to store filenames
    file_names = []
    for filename in os.listdir(input_images_path):
        if filename.endswith(".tif"):
            source = os.path.join(input_images_path, filename)
            destination = os.path.join(output_images_path, filename)
            shutil.copy(source, destination)
            file_names.append(filename)

    # Function to get image annotations
    def get_img_ann(image_id):
        return [ann for ann in data['annotations'] if ann['image_id'] == image_id]

    # Function to get image data
    def get_img(filename):
        return next((img for img in data['images'] if img['file_name'] == filename), None)

    # Iterate through filenames and process each image
    for filename in file_names:
        img = get_img(filename)
        if img is None:
            print(f"Warning: No annotation found for image {filename}")
            continue  # Skip this image if no annotation is found
        
        img_id = img['id']
        img_w = img['width']
        img_h = img['height']
        img_ann = get_img_ann(img_id)

        # Write normalized polygon data to a text file
        if img_ann:
            with open(os.path.join(output_labels_path, f"{os.path.splitext(filename)[0]}.txt"), "a") as file_object:
                for ann in img_ann:
                    current_category = ann['category_id'] - 1
                    polygon = ann['segmentation'][0]
                    normalized_polygon = [format(coord / img_w if i % 2 == 0 else coord / img_h, '.6f') for i, coord in enumerate(polygon)]
                    file_object.write(f"{current_category} " + " ".join(normalized_polygon) + "\n")


# Function to create a YAML file for the dataset
def create_yaml(input_json_path, output_yaml_path, train_path, val_path, test_path=None):
    with open(input_json_path) as f:
        data = json.load(f)
    
    # Extract the category names
    names = [category['name'] for category in data['categories']]
    
    # Number of classes
    nc = len(names)

    # Create a dictionary with the required content
    yaml_data = {
        'names': names,
        'nc': nc,
        'test': test_path if test_path else '',
        'train': train_path,
        'val': val_path
    }

    # Write the dictionary to a YAML file
    with open(output_yaml_path, 'w') as file:
        yaml.dump(yaml_data, file, default_flow_style=False)


In [ ]:

# Function to convert images to YOLO format
def convert_to_yolo(input_images_path, input_json_path, output_images_path, output_labels_path):
    # Open JSON file containing image annotations
    f = open(input_json_path)
    data = json.load(f)
    f.close()

    # Create directories for output images and labels
    os.makedirs(output_images_path, exist_ok=True)
    os.makedirs(output_labels_path, exist_ok=True)

    # List to store filenames
    file_names = []
    for filename in os.listdir(input_images_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp','.tif', '.tiff')):
            source = os.path.join(input_images_path, filename)
            destination = os.path.join(output_images_path, filename)
            shutil.copy(source, destination)
            file_names.append(filename)

    # Function to get image annotations
    def get_img_ann(image_id):
        return [ann for ann in data['annotations'] if ann['image_id'] == image_id]

    # Function to get image data
    def get_img(filename):
        return next((img for img in data['images'] if img['file_name'] == filename), None)

    # Iterate through filenames and process each image
    for filename in file_names:
        img = get_img(filename)
        img_id = img['id']
        img_w = img['width']
        img_h = img['height']
        img_ann = get_img_ann(img_id)

        # Write normalized polygon data to a text file
        if img_ann:
            with open(os.path.join(output_labels_path, f"{os.path.splitext(filename)[0]}.txt"), "a") as file_object:
                for ann in img_ann:
                    current_category = ann['category_id'] - 1
                    polygon = ann['segmentation'][0]
                    normalized_polygon = [format(coord / img_w if i % 2 == 0 else coord / img_h, '.6f') for i, coord in enumerate(polygon)]
                    file_object.write(f"{current_category} " + " ".join(normalized_polygon) + "\n")


In [ ]:
base_input_path = "input/"
base_output_path = "yolo_dataset/"

In [ ]:
# Processing training dataset 
convert_to_yolo(
    input_images_path=os.path.join(base_input_path, "train_images"),
    input_json_path=os.path.join(base_input_path, "train_images/coco_annotations.json"),
    output_images_path=os.path.join(base_output_path, "train/images"),
    output_labels_path=os.path.join(base_output_path, "train/labels")
)

In [ ]:
# Processing validation dataset (if needed)
convert_to_yolo(
    input_images_path=os.path.join(base_input_path, "val_images"),
    input_json_path=os.path.join(base_input_path, "val_images/coco_annotations.json"),
    output_images_path=os.path.join(base_output_path, "valid/images"),
    output_labels_path=os.path.join(base_output_path, "valid/labels")
)

In [ ]:
# Creating the YAML configuration file
create_yaml(
    input_json_path=os.path.join(base_input_path, "train_images/coco_annotations.json"),
    output_yaml_path=os.path.join(base_output_path, "data.yaml"),
    train_path="train/images",
    val_path="valid/images",
    test_path='../test/images'  # or None if not applicable
)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
def display_images_with_annotations(image_paths, annotation_paths):
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    
    for ax, img_path, ann_path in zip(axs.ravel(), image_paths, annotation_paths):
        # Load image using OpenCV and convert it from BGR to RGB color space
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_h, img_w, _ = image.shape
        
        ax.imshow(image)
        ax.axis('off')  # Turn off the axes

        # Open the annotation file and process each line
        with open(ann_path, 'r') as file:
            for line in file:
                parts = line.strip().split()
                category_id = int(parts[0])
                color = tuple(np.random.rand(3))  # Generate a random RGB color
                polygon = [float(coord) for coord in parts[1:]]
                polygon = [coord * img_w if i % 2 == 0 else coord * img_h for i, coord in enumerate(polygon)]
                polygon = [(polygon[i], polygon[i+1]) for i in range(0, len(polygon), 2)]
                patch = patches.Polygon(polygon, closed=True, edgecolor=color, fill=False)
                ax.add_patch(patch)

    plt.tight_layout()
    plt.show()

# Get all image files
image_dir = "yolo_dataset/train/images/"
annotation_dir = "yolo_dataset/train/labels/"
all_image_files = [f for f in os.listdir(image_dir) if f.endswith('.tif')]
random_image_files = random.sample(all_image_files, 4)

# Get corresponding annotation files
image_paths = [os.path.join(image_dir, f) for f in random_image_files]
annotation_paths = [os.path.join(annotation_dir, f.replace(".tif", ".txt")) for f in random_image_files]

In [ ]:
display_images_with_annotations(image_paths, annotation_paths)

In [4]:
from ultralytics import YOLO
from IPython.display import display, Image
from IPython import display
display.clear_output()
!yolo checks

Ultralytics YOLOv8.2.42 🚀 Python-3.8.10 torch-2.2.1+cu121 CUDA:0 (Quadro RTX 8000, 48585MiB)
Setup complete ✅ (72 CPUs, 250.5 GB RAM, 1870.7/1876.2 GB disk)

OS                  Linux-5.15.0-107-generic-x86_64-with-glibc2.29
Environment         Linux
Python              3.8.10
Install             pip
RAM                 250.54 GB
CPU                 Intel Xeon Gold 5220 2.20GHz
CUDA                12.1

numpy               ✅ 1.24.4<2.0.0,>=1.23.5
matplotlib          ✅ 3.7.5>=3.3.0
opencv-python       ✅ 4.8.0.74>=4.6.0
pillow              ✅ 10.2.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.10.1>=1.4.1
torch               ✅ 2.2.1>=1.8.0
torchvision         ✅ 0.17.1>=0.9.0
tqdm                ✅ 4.66.2>=4.64.0
psutil              ✅ 5.9.8
py-cpuinfo          ✅ 9.0.0
pandas              ✅ 2.0.3>=1.1.4
seaborn             ✅ 0.13.2>=0.11.0
ultralytics-thop    ✅ 2.0.0>=2.0.0


In [ ]:
model = YOLO("yolov8m-seg.yaml") # build a new model from yaml
model = YOLO("yolov8m-seg.pt") # transfer weights fro a pre-trained model (recommended for training)

In [ ]:
with open('yolo_dataset/data.yaml', 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
sanofi_projct = "yolo_dataset/results"
name = "125_epochs_seg_m"

In [ ]:
# Train the model
results = model.train(data='yolo_dataset/data.yaml',
                    project=sanofi_projct,
                    name=name,
                    epochs=125
                    )
     

In [ ]:
def evaluation(model, data_dir, ious):

    #ious = np.linspace(.5,1,50)
    precision = []
    recall = []
    jac_idx = []
    for io in ious:
        tp_eval = YOLO(model)
        metrics = tp_eval.val(data= data_dir,
                              iou=io,
                              device=0
                              )
        precision.append(metrics.results_dict["metrics/precision(B)"])
        recall.append(metrics.results_dict["metrics/recall(B)"])
        conf_mtrx = metrics.confusion_matrix
        j_idx = conf_mtrx.matrix[0][0]/conf_mtrx.matrix.sum()
        jac_idx.append(j_idx)
    return jac_idx, precision, recall

In [ ]:
jac_idx, precision, recall = evaluation(model="yolo_dataset/results/125_epochs_seg_l2/weights/best.pt", data_dir="yolo_dataset/data.yaml", ious=np.linspace(.5,1,50))


In [6]:
model1 = YOLO("yolo_dataset/results/125_epochs_seg_l2/weights/best.pt")

image = "/home/kamenan/Documents/Stage Dev Deep Learning/YOLOV8 evaluation/sanofi_test/test_img/200L_GBD_50K_029.dm3.tif.tif"

results = model1.predict(image, conf=0.5, show_labels=True, show_boxes=True, show=True)  #Adjust conf threshold


image 1/1 /home/kamenan/Documents/Stage Dev Deep Learning/YOLOV8 evaluation/sanofi_test/test_img/200L_GBD_50K_029.dm3.tif.tif: 640x640 3 Archs, 3 Blackberrys, 24 Rounds, 24.2ms
Speed: 5.3ms preprocess, 24.2ms inference, 154.4ms postprocess per image at shape (1, 3, 640, 640)


: 

In [ ]:
import cv2
import numpy as np
# Load the image
try:
    img = cv2.imread("/home/kamenan/Documents/Stage Dev Deep Learning/YOLOV8 evaluation/sanofi_test/test_img/200L_GBD_50K_029.dm3.tif.tif")
    h, w, c = img.shape
except cv2.error as e:
    print(f"Failed to load image: {e}")
    exit()

# Create the YOLOv8 model
try:
    model = YOLO("yolo_dataset/results/125_epochs_seg_l2/weights/best.pt")
except Exception as e:
    print(f"Failed to load model: {e}")
    exit()

# Perform the prediction
img2 = cv2.resize(img, (640,640))
results = model.predict(img2, iou=.5)

if(results[0].masks is not None):
    # Get the size of the original image (height, width, channels)
    h2, w2, c2 = results[0].orig_img.shape
    print((w2, h2))

    # Create a black image with the same size as the original image
    black_img = np.zeros_like(results[0].orig_img)
    black_img = black_img[:,:,0]
    print(black_img.shape)
    # Create a copy of the original image to layer the masks on
    layered_img = results[0].orig_img.copy()

    # Loop over all masks in the results
    for i, mask_raw in enumerate(results[0].masks):
        # Convert mask to single channel image
        mask_raw = mask_raw.cpu().data.numpy().transpose(1, 2, 0)

        # Resize the mask to the same size as the image (can probably be removed if image is the same size as the model)
        mask = cv2.resize(mask_raw, (640, 640))

        # Convert the mask to the correct data type
        mask = mask.astype(np.uint8)
        #print(mask.shape)

        #multiply by i+1
        mask = mask*(i+1)

        #add mask to black_img
        black_img = np.maximum(black_img, mask)
        #black_img = black_img + mask
        
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(0) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        exit()

# Close all windows
#cv2.imwrite("mask_raw.png", mask_raw)
#cv2.imwrite("mask.png", mask)
black_img = cv2.resize(black_img, (w,h), interpolation=cv2.INTER_NEAREST)
cv2.imwrite("black_img.png", black_img)
cv2.destroyAllWindows()

In [ ]:
def seg_masks(data_dir: str, image_dir: str, model: str) -> None:

    """
    Get the segmented masks with a given infered model

    Params:
    -------
           data_dir: Absolute path to image to segment
           image_dir: relative path to image to segment
           model: Model to make the inference
    """


    # set the current directory to the image directory
    #os.chdir(image_dir)

    # create segmented masks directory
    seg_masks_dir = os.path.join(data_dir, 'seg_masks')
    if not os.path.exists(seg_masks_dir):
        try:
            os.makedirs(seg_masks_dir, exist_ok=True)
        except os.error as e:
            print(f'Same directory exist:{e}')
            exit()
    # set the path to the image directory
    img_dir = os.path.join(data_dir, image_dir)

    # iterate over all files in the test image directory
    for file in os.listdir(img_dir):
        # contruct full file path
        file_path = os.path.join(img_dir, file)

        # check if the file is image
        if os.path.isfile(file_path) and file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp','.tif', '.tiff')):
            # read the image with OpenCV
            try:
                img = cv2.imread(file_path)
                h, w, _ = img.shape
            except cv2.error as e:
                print(f"Failed to load image: {e}")
                exit()

        # create the YOLOV8 
        try:
            mdel = YOLO(model)
        except Exception as e:
            print(f"Failed to load model: {e}")
            exit()
        
        # Perform the prediction
        img2 = cv2.resize(img, (640,640))
        #img2 = cv2.resize(img, (img.shape[1], img.shape[0]))
        #imgsz = (img2.shape[0], img2.shape[0])
        results = mdel.predict(img2, iou=.5)

        if(results[0].masks is not None):
            # Get the size of the original image (height, width, channels)
            h2, w2, c2 = results[0].orig_img.shape

            # Create a black image with the same size as the original image
            black_img = np.zeros_like(results[0].orig_img)
            black_img = black_img[:,:,0]
            
            # Create a copy of the original image to layer the masks on
            #layered_img = results[0].orig_img.copy()

            # Loop over all masks in the results
            for i, mask_raw in enumerate(results[0].masks):
                # Convert mask to single channel image
                mask_raw = mask_raw.cpu().data.numpy().transpose(1, 2, 0)

                # Resize the mask to the same size as the image (can probably be removed if image is the same size as the model)
                mask = cv2.resize(mask_raw, (w2, h2))

                # Convert the mask to the correct data type
                mask = mask.astype(np.uint16)

                #multiply by i+1
                mask = mask*(i+1)

                #add mask to black_img
                black_img = np.maximum(black_img, mask)

            # Break the loop if 'q' key is pressed
            if cv2.waitKey(0) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                exit()
        
        name, ext = os.path.splitext(file)

        # set the path to the segmented mask with the same naming system like images
        seg_mask = os.path.join(seg_masks_dir, seg_masks_dir + '/' + name.replace("img", "masks") + "_seg." + ext)
        
        # Close all windows
        black_img = cv2.resize(black_img, (w,h), interpolation=cv2.INTER_NEAREST)
        cv2.imwrite(seg_mask, black_img)
        cv2.destroyAllWindows()